# Role R Likelihood-free Simulation Based Inference

In [1]:
from roler.model import ModelPrior
from roler.distributions import *

In [3]:
prior = ModelPrior(
    individuals_local=IntDistribution(50, 300),
    individuals_meta=IntDistribution(400, 1000),
    species_meta=50,
    speciation_local=0.05,
    speciation_meta=0.05,
    extinction_meta=0.05,
    env_sigma=0.5,
    trait_sigma=1,
    comp_sigma=0.5,
    dispersal_prob=0.1,
    mutation_rate=0.01,
    equilib_escape=1,
    num_basepairs=250,
    init_type='oceanic_island',
    niter=2000,
    niterTimestep=10
)

## Generating a Dataset of Simulations

This cell generates a dataset of simulation results using the `Dataset` class. It initializes a `Dataset` object with the previously defined `simulator` and `prior`. The `generate_dataset` method is then called to generate `samples=10` simulation runs. The resulting parameter values (`theta`) and summary statistics (`x`) are stored. Finally, the shapes of `theta` and `x` are printed to show the dimensions of the generated dataset.

In [4]:
from roler.datasets import Dataset
from roler.simulation import Simulator

simulator = Simulator()
dataset = Dataset(simulator=simulator, prior=prior)
theta, x = dataset.generate_dataset(samples=10)

print("theta.size", theta.shape)
print("x.size", x.shape)

Installing the 'roleR' R package from GitHub...


R[write to console]: Using GitHub PAT from the git credential store.

R[write to console]: Skipping install of 'roleR' from a github remote, the SHA1 (cc6546a1) has not changed since last install.
  Use `force = TRUE` to force installation



theta.size torch.Size([1995, 2])
x.size torch.Size([1995, 9])


## Performing Inference with SNPE (Sequential Neural Posterior Estimation)

This cell uses the `sbi` library to perform inference and estimate the posterior distribution of the model parameters given the simulated data.

1.  An `SNPE` object is initialized using the joint uniform prior obtained from `prior.get_joint_uniform()`.
2.  The simulated parameter values (`theta`) and summary statistics (`x`) generated in the previous cell are used to train a neural density estimator using `snpe.append_simulations(theta, x).train()`.
3.  A posterior object is built from the trained density estimator using `snpe.build_posterior(density_estimator)`. This posterior object can then be used to sample from the approximate posterior distribution or evaluate its density.

In [5]:
from sbi.inference import SNPE

snpe = SNPE(prior=prior.get_joint_uniform())
density_estimator = snpe.append_simulations(theta, x).train()
posterior = snpe.build_posterior(density_estimator)

/Users/michelleuy/Desktop/github/likelihood-free-inference/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


 Neural network successfully converged after 314 epochs.

## Evaluating the Learned Posterior with Observed Data

This cell evaluates the performance of the learned posterior by:

1.  Generating a single "observed" dataset point `x_obs` with corresponding true parameter values `theta_true` using `dataset.generate_dataset(samples=1)`.
2.  Sampling from the posterior distribution conditioned on the observed data `x_obs` using `posterior.sample((1000,), x=x_obs)`, creating 1000 posterior samples.
3.  Computing the posterior mean as a point estimate of the parameters.
4.  Printing the true parameter values (`theta_true`) and the posterior mean estimate for comparison. This allows you to assess how well the inference procedure recovers the true parameters given the observed data.

In [6]:
theta_true, x_obs = dataset.generate_dataset(samples=1)
theta_true, x_obs = theta_true[-1], x_obs[-1]
print("Observed simulation output:", x_obs)

# Use the learned posterior to sample inferred parameters given the observed output
posterior_samples = posterior.sample((1000,), x=x_obs)
print("Posterior samples shape:", posterior_samples.shape)

# Compute a point estimate (e.g. the posterior mean)
posterior_mean = posterior_samples.mean(dim=0)
print("Posterior mean estimate:", posterior_mean)

print()
print("Theta True      :", theta_true)
print("Theta Prediction:", posterior_mean)

Observed simulation output: tensor([15.3946, 12.5519, 10.9761, 10.0596, 14.6334, 12.1218, 10.8196, 10.0883,
        20.0000])


Drawing 1000 posterior samples: 1097it [00:00, 53405.51it/s]            

Posterior samples shape: torch.Size([1000, 2])
Posterior mean estimate: tensor([104.0333, 760.6565])

Theta True      : tensor([ 55.3132, 778.7090])
Theta Prediction: tensor([104.0333, 760.6565])


In [7]:
print("True       :", dataset.get_params_from_tensor(theta_true))
print("Prediction :", dataset.get_params_from_tensor(posterior_mean))

True       : ModelParams(individuals_local=55, individuals_meta=779, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
Prediction : ModelParams(individuals_local=104, individuals_meta=761, species_meta=50, speciation_local=0.05, speciation_meta=0.05, extinction_meta=0.05, env_sigma=0.5, trait_sigma=1, comp_sigma=0.5, dispersal_prob=0.1, mutation_rate=0.01, equilib_escape=1, num_basepairs=250, init_type='oceanic_island', niter=2000, niterTimestep=10)
